In [2]:
# !pip install fosforml

In [2]:
# pip show fosforml

## Sklearn Models Registration

### session creation

In [70]:
from fosforml.model_manager.snowflakesession import get_session, get_connection_params
session = get_session()
connection_params = get_connection_params()

### modeling

#### Multiclass classification

In [71]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_iris() 
feature_names = m_dataset.feature_names
u_features = [col.replace(" ","_").replace("(","").replace(")","") for col in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

## Custom Function

In [157]:
class CustomModelScore:
    
    @staticmethod
    def pre_process(input_data):
        return self.input_df

# def score_func(model,input_dataframe:pd.DataFrame) -> pd.DataFrame:
#     return input_dataframe

In [164]:
from snowflake.ml.model import custom_model
from snowflake.ml.model.custom_model import ModelContext

mc = custom_model.ModelContext(
    artifacts={
    },
    models={
        'm1': model,
        'feature_preproc' : CustomModelScore
    }
)

In [171]:
class ExamplePipelineModel(custom_model.CustomModel):
    def __init__(self, context: ModelContext) -> None:
        super().__init__(context)
        # self.input_cols = x_train.columns.tolist()
        # self.output_cols = ['Prediction']
        # self.label_cols = ['Target']
        
    @custom_model.inference_api
    def predict(self, input_data: pd.DataFrame) -> pd.DataFrame:
        print(dir(self.context.model_ref('m1')))
        model_output = self.context.model_ref('m1').predict(input_data)
        return pd.DataFrame({
            'output': model_output })

In [172]:
my_model_pipeline = ExamplePipelineModel(mc)

In [173]:
my_model_pipeline.predict(x_test).head()

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_model', '_name', 'model', 'name']


,output
0,0
1,0
2,2
3,0
4,0


In [94]:
from snowflake.ml.registry import Registry
reg = Registry(session=session, database_name="INSIGHT_DESIGNER_SPCS", schema_name="INSIGHT_SPCS_SCHEMA")
mv = reg.log_model(my_model_pipeline,
            model_name="my_custom_model_pipeline",
            version_name="v3",
            conda_dependencies=["scikit-learn"],
            comment="My Custom ML Model Pipeline",
            sample_input_data=x_train)

In [80]:
# reg.show_models()

In [81]:
output_df = mv.run(session.create_dataframe(x_train))

SnowparkSQLException: (1300) (1304): 01b6026c-0607-d638-0072-f3030fa85bba: 002003 (42S02): SQL compilation error:
Object 'INSIGHT_DESIGNER_SPCS.INSIGHT_SPCS_SCHEMA.SNOWPARK_TEMP_TABLE_XKIAQTRJ15' does not exist or not authorized.

In [ ]:
output_df.to_pandas()

In [ ]:
# my_model_pipeline.

## Model registration

In [66]:
str(type(my_model_pipeline))

<class '__main__.ExamplePipelineModel'>


In [61]:
from fosforml import register_model
register_model(
  model_obj=my_model_pipeline,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RandomMulticlassMOdelForTesting",
  dataset_source="Snowflake",
  name="RandomMultieefefefclassMOdelForTesting16July",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Exception: Failed to get model performance metrics. 'feature_names'